<a href="https://colab.research.google.com/github/ujju0906/Digitisation-and-automation-of-Rajasthan-FIR-documents-using-google-vision-api-/blob/main/Digitisation_of_fir_documents_using_google_vision_and_extraction_of_data_using_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/sample_data')

In [ ]:
import pandas as pd


In [ ]:
import numpy as num

In [ ]:
from google.cloud import storage

In [ ]:
!pip install google-cloud-vision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.cloud import vision

In [ ]:
import json
import re

In [ ]:
import re
import os 
import json
import sys

#gs://fir_files/27564006150123.pdf
# Setting credentials using the downloaded JSON file
batch_size = 40
mime_type = 'application/pdf'

path = 'fir.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
storage_client = storage.Client.from_service_account_json(path)


# Creating bucket object

bucket = storage_client.get_bucket('fir_files')

filename = '27564033090156.pdf'
# Name of the object to be stored in the bucket

object_name_in_gcs_bucket = bucket.blob(filename)

# Name of the object in local file system

object_name_in_gcs_bucket.upload_from_filename(filename)
client = vision.ImageAnnotatorClient()

feature = vision.Feature(
    type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

gcs_source_uri = 'gs://fir_files/'+filename
gcs_source = vision.GcsSource(uri=gcs_source_uri)
input_config = vision.InputConfig(
    gcs_source=gcs_source, mime_type=mime_type)

gcs_destination_uri = 'gs://fir_files/' + filename +'extttt'
gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)
output_config = vision.OutputConfig(
    gcs_destination=gcs_destination, batch_size=batch_size)

async_request = vision.AsyncAnnotateFileRequest(
    features=[feature], input_config=input_config,
    output_config=output_config)

operation = client.async_batch_annotate_files(
    requests=[async_request])

print('Waiting for the operation to finish.')
operation.result(timeout=180)


storage_client = storage.Client()

match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
bucket_name = match.group(1)
prefix = match.group(2)

bucket = storage_client.get_bucket(bucket_name)


blob_list = list(bucket.list_blobs(prefix=prefix))
print('Output files:')
for blob in blob_list:
    print(blob.name)


output = blob_list[0]

json_string = output.download_as_string()
response = json.loads(json_string)
fir_text =""
for i in response['responses']:
  fir_text = fir_text + i['fullTextAnnotation']['text']


print('Full text:\n')
print(fir_text)



Waiting for the operation to finish.
Output files:
27564033090156.pdfexttttoutput-1-to-18.json
Full text:

रा.मु.ज. / 2-2009 / 2000 बुक
बुक नं.
7093
1.
(iv)
3. (a) (क)
District (जिला) पाली P.S. (थाना सिर मारी Gear (वर्ष) 2009 FIR No. (प्र.सू. रि.सं.) 156.
2. (i)
Act (अधिनियम) भा.द.स.
Sections (धाराएँ)
(ii)
Act (अधिनियम)
Sections (धाराएँ)
Act (अधिनियम)
Sections (धाराएँ)
Other Acts & Sections (अन्य अधिनियम एवं धाराएँ)
Occurrence of offence (घटना का) Day (दिन)
Time Period (पहर) शाम Time from (बजे से) 6pm
DIGESTED
W
Co.Sojat
कार्यालय उप अधीक्षक
क्रमांक..
ਜੀਜ
7
CG1. लकित 4995
1/11/09
(b) (ख)
(c) (ग)
14. Type of Information (सूचना कैसे प्राप्त हुई )
(c) (ग)
80.34....
461/mey
वत सोनमती (पाली Address (पता)
FIRST
(2)
(3).
INFORMATION REPORT
प्रथम सूचना रिपोर्ट
02
(Under Section 154 Cr. P.C.)
(दण्ड प्रक्रिया संहिता धारा 154 के अन्तर्गत)क्रम सं.
Time (समय). 6.30pm
Time (समय) 6:30pm
Written/Oral (लिखित/मौखिक)
156 (3) COM
Place of Occurrence : घटनास्थल का ब्यौरा (a) (क) Direction and distance from 

In [ ]:
fir_text = fir_text.replace('\n',"")

In [ ]:
fir_text

'रा.मु.ज. / 2-2009 / 2000 बुकबुक नं.70931.(iv)3. (a) (क)District (जिला) पाली P.S. (थाना सिर मारी Gear (वर्ष) 2009 FIR No. (प्र.सू. रि.सं.) 156.2. (i)Act (अधिनियम) भा.द.स.Sections (धाराएँ)(ii)Act (अधिनियम)Sections (धाराएँ)Act (अधिनियम)Sections (धाराएँ)Other Acts & Sections (अन्य अधिनियम एवं धाराएँ)Occurrence of offence (घटना का) Day (दिन)Time Period (पहर) शाम Time from (बजे से) 6pmDIGESTEDWCo.Sojatकार्यालय उप अधीक्षकक्रमांक..ਜੀਜ7CG1. लकित 49951/11/09(b) (ख)(c) (ग)14. Type of Information (सूचना कैसे प्राप्त हुई )(c) (ग)80.34....461/meyवत सोनमती (पाली Address (पता)FIRST(2)(3).INFORMATION REPORTप्रथम सूचना रिपोर्ट02(Under Section 154 Cr. P.C.)(दण्ड प्रक्रिया संहिता धारा 154 के अन्तर्गत)क्रम सं.Time (समय). 6.30pmTime (समय) 6:30pmWritten/Oral (लिखित/मौखिक)156 (3) COMPlace of Occurrence : घटनास्थल का ब्यौरा (a) (क) Direction and distance from P.S. (थाने से दिशा एवं दूरी) बच्ख उत्तरपुलिसनफासला 6KBeat No. (बीट सं.) 1VCNB No. S पार्ट DalSNo. 247/09out Unter6. Complainant / Informant (शिकायतकर्ता

In [ ]:
t = re.search('if required(.*)Action taken ',fir_text)

In [ ]:
t.group(0)

'if required (यदि आवश्यक हो, तो अलग पृष्ठ नत्थी करेंसेवा मेंI.I.F.-I./ एकीकृत जच फार्म-1पृष्ठ सं. -2Xश्रीमान न्यायिक मजिस्ट्रेट साहबpilभारवाड़ जंक्शन30मुलजिम चालक वाहन जीप नम्बर आर. जे. 19/4709दलपतराज पुत्र श्री गंगाराम जी जाति मेगवाल निवासीपाली दरवाजा के बाहर सोजत फिटी पी.एम. सोजत सिटीजिला- पाली (राज.)इस्तगासा अन्तर्गत धारा 279,337.338 आई.पी.सी.इस्तगासा मुस्तगीस की ओर से निम्न है।●● यह कि मुस्तमोस का छोटा भाई बाल किशन जो दि. 28.9.०१ कोशाम को करीब 6बजे अपनी हिरो होण्डा मोटर साईकल से गोगाजी केमंदिर बाघारी परदर्शन के लिये जा रहा था। चूंकि मुस्त गीत काभाई बाल फिसन अपने खसूर से मिलने के लिए वोपरी जाना- चाहता था, इसलिये सुस्तगीस का भाई अपनी मोटर साइकलसहितमेलाप चौराहे पर पड़ा था कि इतने में सामने से वाहन जीप नम्बरआर. जे. 19/4709 को उसका चालक बहुत ही तेजगति व लापरवाही पूर्वकजीप को चलाता हुआ आया एवंव बाल किसन केलगातारLI.F.I./एकीकृत जाँच फार्म-।कपृष्ठ से - 3परके सही मंदिर में पड़े हुए के मोटर साईकल के जोरदार टक्कर मारी इसदुर्घटना में मुस्तगीस के भाई बाल किसन की राग डर गई व मौकेबेहोश होकर गिर गया। 

In [ ]:
z = len(response['responses'])-1
x = response['responses'][z]
y = x['fullTextAnnotation']['text']

In [ ]:
y

'Acharya Shree Raghunath Smiriti Jain\nGOVT. HOSPITAL, SOJAT CITY\nRAJASTHAN MEDICARE RELIEF SOCIETY, SOJAT\nX-Ray Report\nNAME .......….... angalal.com\nX-Ray........\nMcCxRoy Bal Kishan 20 Mangalalov\nMy -\nscar RE- Cres.\nMacky No. 4134 Rt. Leg AP 2 let-vi-end\nTofal one mac\nAGE\nDate\n289\nShilos\n5 hilo?\nmaciley adviced and done in ~ forerence This Rusher\nKey refert there is # of Telia fhufa. & KNail at sita.\nSovewing seen. I caller at the scruting site\nthem is old Heale? fracture.\nalso preven\nI union\nof fivela is\n19\ncepen. - the myths is graviors of more then 14 days Duation.\n00\n11-08'

In [ ]:
if(t):
   t = t.group(0)
   print(t)

Status of the accused :Forwarded / Bailed by police / Bailed hy court / Judicial custody / Absconding / Proclaimed offender(tick / applicable portion)अभियुक्त की स्थिति :पेश किया गया / पुलिस द्वारा पर / न्यायालय द्वारा जमानत पर / न्यायिक हिरासत में / फरार / घोषित अपराधी(लागू हिस्से पर '' का चिन्ह लगाएँ)131Smeह(2)लस थाना परिमार1+11-मा-पाणी (राज.)ser 72584 (3)@B-TAINA 1981 SCEAOBE-122ian CS)of2. Particulars of accused persons not charge-sheeted (suspect): (Use separate sheet for each suspect)आरोप पत्र दाखिल न किए गए (संदिग्ध) अभियुक्तों का विवरण (प्रत्येक संदिग्ध के लिए अलग पृष्ठ नत्थी करें)SI. No. ( क्रम संख्या)..Name (नाम)..Father's / Husband's Name(पिता/पति का नाम)..(i)(iv) Sex (लिंग)...…...…..(vii) Religion (धर्म).(ix) Occupation (व्यवसाय).(x) Address ( पता)..Date of Issue ( जारी करने की तिथि )....(iii) Date/Year of Birth (जन्मतिथि / वर्ष )...(vi) Passport No. (पासपोर्ट स.)..Place of Issue (जारी करने का स्थान).... (vii) Whether SC/ST/OBC (अनु. जाति / अनु. जनजाति / अन्य पिछड़े वर्ग)..

In [ ]:
t= re.search('contents(.*)Action taken',fir_text)

In [ ]:
t.group(0)

'contents (प्र.सू.रि की विषय वस्तु) Attach separate sheet, if required (यदि आवश्यक हो, तो अलग पृष्ठ नत्थी करेंसेवा मेंI.I.F.-I./ एकीकृत जच फार्म-1पृष्ठ सं. -2Xश्रीमान न्यायिक मजिस्ट्रेट साहबpilभारवाड़ जंक्शन30मुलजिम चालक वाहन जीप नम्बर आर. जे. 19/4709दलपतराज पुत्र श्री गंगाराम जी जाति मेगवाल निवासीपाली दरवाजा के बाहर सोजत फिटी पी.एम. सोजत सिटीजिला- पाली (राज.)इस्तगासा अन्तर्गत धारा 279,337.338 आई.पी.सी.इस्तगासा मुस्तगीस की ओर से निम्न है।●● यह कि मुस्तमोस का छोटा भाई बाल किशन जो दि. 28.9.०१ कोशाम को करीब 6बजे अपनी हिरो होण्डा मोटर साईकल से गोगाजी केमंदिर बाघारी परदर्शन के लिये जा रहा था। चूंकि मुस्त गीत काभाई बाल फिसन अपने खसूर से मिलने के लिए वोपरी जाना- चाहता था, इसलिये सुस्तगीस का भाई अपनी मोटर साइकलसहितमेलाप चौराहे पर पड़ा था कि इतने में सामने से वाहन जीप नम्बरआर. जे. 19/4709 को उसका चालक बहुत ही तेजगति व लापरवाही पूर्वकजीप को चलाता हुआ आया एवंव बाल किसन केलगातारLI.F.I./एकीकृत जाँच फार्म-।कपृष्ठ से - 3परके सही मंदिर में पड़े हुए के मोटर साईकल के जोरदार टक्कर मारी इसदुर्घटना में मुस्तग

In [ ]:
u ='rifnirfnirfni45ncininccincicinicnicinc'
t = re.search('nci(.*)inc',u)

In [ ]:
t.group(0)

'ncininccincicinicnicinc'

In [ ]:
fir_text =fir_text.replace(' ','')

In [ ]:
fir_text

'रा.मु.ज./2-2009/2000बुकबुकनं.70931.(iv)3.(a)(क)District(जिला)पालीP.S.(थानासिरमारीGear(वर्ष)2009FIRNo.(प्र.सू.रि.सं.)156.2.(i)Act(अधिनियम)भा.द.स.Sections(धाराएँ)(ii)Act(अधिनियम)Sections(धाराएँ)Act(अधिनियम)Sections(धाराएँ)OtherActs&Sections(अन्यअधिनियमएवंधाराएँ)Occurrenceofoffence(घटनाका)Day(दिन)TimePeriod(पहर)शामTimefrom(बजेसे)6pmDIGESTEDWCo.Sojatकार्यालयउपअधीक्षकक्रमांक..ਜੀਜ7CG1.लकित49951/11/09(b)(ख)(c)(ग)14.TypeofInformation(सूचनाकैसेप्राप्तहुई)(c)(ग)80.34....461/meyवतसोनमती(पालीAddress(पता)FIRST(2)(3).INFORMATIONREPORTप्रथमसूचनारिपोर्ट02(UnderSection154Cr.P.C.)(दण्डप्रक्रियासंहिताधारा154केअन्तर्गत)क्रमसं.Time(समय).6.30pmTime(समय)6:30pmWritten/Oral(लिखित/मौखिक)156(3)COMPlaceofOccurrence:घटनास्थलकाब्यौरा(a)(क)DirectionanddistancefromP.S.(थानेसेदिशाएवंदूरी)बच्खउत्तरपुलिसनफासला6KBeatNo.(बीटसं.)1VCNBNo.Sपार्टDalSNo.247/09outUnter6.Complainant/Informant(शिकायतकर्ता/इत्तिलादेनेवाला):(इत्तिलादेनेवाला):(a)(क)Name(नाम)श्रीदलपतराज..(b)(ख)Father\'s/Husband\'sName(पिता/पतिकानाम)(c)(ग)(e)(ड)मेलाप

In [ ]:
a = 19
a = -a
a

-19